In [1]:
# basic python data handling analysis modules
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set()
import pickle, os, gc, re
# small utility functions
from utility import *

# interactive jupyter widgets!
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%matplotlib inline

In [163]:
# import warnings
# warnings.filterwarnings('ignore')
# # cleanliness!

In [162]:
Dataset_directory = ".."+os.sep+"Datasets"+os.sep
tmdb_dir = create_subdir(Dataset_directory, "themoviedb")

In [164]:
# import datetime
# datetime.datetime.now()

In [ ]:
# https://developers.themoviedb.org/3/getting-started/daily-file-exports

# The export job runs every day starting at around 7:00 AM UTC, and all files are available by 8:00 AM UTC.

# http://files.tmdb.org/p/exports/movie_ids_04_28_2017.json.gz

In [28]:
import datetime

utc_now = datetime.datetime.utcnow()

if utc_now.hour<8: # upload for 'today' not necessary donw yet
    utc_now = today - datetime.timedelta(days = 1)
    
movie_ids_file = "http://files.tmdb.org/p/exports/movie_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
movie_ids_file

'http://files.tmdb.org/p/exports/movie_ids_06_26_2019.json.gz'

In [76]:
import shutil
import tempfile
import urllib.request
import gzip

with urllib.request.urlopen(movie_ids_file) as response:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        shutil.copyfileobj(response, tmp_file)

# with open(tmp_file.name) as html:
#     decompressedFile = gzip.GzipFile(fileobj=html, mode='rb')
#     outFilePath = tmdb_dir + movie_ids_file.split("/")[-1].replace(".gz","")
#     with open(outFilePath, 'w', encoding="utf8") as outfile:
#         outfile.write(decompressedFile.read( 'rt' ))

#     pass


with gzip.open(tmp_file.name, mode='rt', encoding="utf-8") as f_in:
    with open(outFilePath, 'w', encoding="utf-8") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [82]:
movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467223,467223.000000,467223,467223.000000,467223
unique,2,NaN,429550,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443083,NaN,46,NaN,444777
mean,NaN,331788.498929,NaN,1.111966,NaN
std,NaN,175573.963916,NaN,2.037446,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191249.000000,NaN,0.600000,NaN
50%,NaN,346870.000000,NaN,0.600000,NaN
75%,NaN,481636.500000,NaN,0.778000,NaN


In [83]:
!pip install tmdbsimple

In [ ]:
# http://api.themoviedb.org/3/movie/16535?api_key=###

In [165]:
# import tmdbsimple as tmdb
text_file = open(tmdb_dir+"my_api_key.txt", "r")
# tmdb.API_KEY = text_file.readlines()[0]
api_key = text_file.readlines()[0]

In [98]:
# with urllib.request.urlopen("http://api.themoviedb.org/3/movie/16535?api_key="+tmdb.API_KEY) as response:
#     resp = response.readlines()
# #     with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
# #         shutil.copyfileobj(response, tmp_file)

In [155]:
import json
import time

tmdb_df = pd.DataFrame(columns = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'])

In [ ]:
count = 0
fail_ids = []
for tmdb_id in movie_ids["id"][1200:].values:
    print(tmdb_id)
    try:
        with urllib.request.urlopen("http://api.themoviedb.org/3/movie/"+str(tmdb_id)+"?api_key="+api_key) as response:
            resp = response.readlines()[0]
        # just assuming it doesn't break!

        tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(resp), orient='index').T.values[0]
    except:
        print("failure at id = "+str(tmdb_id))
        fail_ids.append(tmdb_id)
        
    count = count +1
    if (count % 30) == 0:
        time.sleep(10) # respect the rate timer! no more than 40 requests every 10s
        
tmdb_df.to_msgpack( tmdb_dir+"tmdb_df"+".msgpack")

9594
9595
9596
9597
9598
9599
9600
10201
10202
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
10217
10218
10219
10220
10221
10222
10223
10224
10225
10226
10227
10228
10229
10231
10232
10234
10235
10236
10237
10238
10239
10241
10242
10243
10244
10246
10247
10248
10249
10250
10251
10252
10253
10254
10255
10256
10257
10258
10259
10260
10261
10263
10264
10265
10266
10267
10268
10269
10270
10271
10272
10273
10274
10275
10276
10278
10279
10280
10281
10282
10283
10285
10286
10287
10288
10289
10291
10292
10293
10294
10295
10296
10297
10299
10300
10301
10303
10304
10305
10306
10307
10308
10309
10310
10311
10312
10313
10314
10315
10316
10317
10319
10320
10322
10323
10324
10326
10327
10328
10329
10330
10331
10333
10335
10336
10337
10338
10339
10340
10341
10342
10344
10345
10347
10348
10349
10350
10351
10353
10354
10355
10356
10357
10358
10359
10360
10361
10362
10363
10364
10366
10367
10368
10369
10370
10371
10372
10373
10374
10375
10376
10377
10378
10379
10380
10381

16774
16780
16781
16783
16784
16785
16788
16789
16790
16791
16800
17401
17402
17403
17407
17408
17409
17413
17414
17415
17417
17418
17421
17422
17428
17430
17431
17432
17433
17435
17436
17439
17443
17445
17447
17448
17450
17452
17455
17456
17457
17460
17464
17465
17466
17467
17469
17470
17473
17474
17478
17479
17480
17481
17482
17483
17485
17486
17487
17494
17495
17496
17499
17501
17507
17508
17509
17510
17511
17512
17513
17514
17515
17516
17517
17518
17519
17520
17521
17522
17523
17524
17525
17526
17527
17528
17529
17530
17532
17533
17534
17535
17536
17537
17538
17539
17540
17541
17542
17543
17544
17545
17546
17547
17548
17549
17550
17551
17552
17553
17554
17555
17556
17557
17558
17559
17561
17562
17566
17571
17575
17577
17578
17580
17581
17582
17584
17585
17586
17589
17590
17591
17592
17593
17594
17595
17599
17600
18203
18204
18205
18206
18208
18209
18210
18211
18212
18213
18214
18215
18216
18217
18218
18219
18220
18221
18222
18223
18224
18225
18228
18229
18230
18231
18232
18238
1823

26292
26293
26294
26295
26296
26298
26299
26300
26301
26302
26303
26304
26305
26306
26309
26311
26312
26314
26316
26317
26318
26319
26320
26321
26323
26325
26326
26327
26328
26329
26331
26332
26333
26334
26335
26336
26337
26338
26339
26340
26341
26342
26344
26346
26347
26349
26350
26351
26352
26353
26354
26355
26356
26357
26358
26359
26360
26361
26362
26363
26364
26365
26366
26367
26368
26369
26370
26371
26372
26373
26374
26376
26378
26379
26380
26381
26385
26386
26387
26388
26389
26390
26391
26393
26398
26400
27001
27003
27004
27005
27006
27007
27009
27010
27011
27012
27016
27017
27018
27019
27020
27022
27023
27027
27028
27029
27030
27031
27033
27034
27035
27036
27037
27039
27040
27042
27043
27045
27046
27047
27052
27053
27056
27057
27058
27059
27060
27061
27062
27063
27064
27065
27066
27067
27069
27070
27072
27073
27074
27076
27079
27083
27084
27085
27088
27089
27090
27091
27092
27093
27094
27095
27098
27099
27100
27101
27102
27103
27104
27105
27106
27107
27108
27111
27112
27113
2711

33540
33541
33542
33543
33544
33545
33546
33548
33551
33554
33555
33556
33557
33558
33560
33563
33564
33565
33569
33570
33571
33572
33575
33576
33578
33579
33580
33581
33582
33583
33584
33586
33587
33588
33589
33591
33592
33598
33600
34203
34204
34205
34207
34208
34209
34210
34211
34212
34213
34214
34216
34219
34220
34222
34223
34226
34228
34229
34231
34233
34243
34247
34269
34275
34276
34277
34279
34280
34282
34283
34288
34295
34297
34299
34300
34302
34303
34305
34306
34308
34309
34311
34312
34314
34317
34318
34319
34320
34321
34322
34324
34325
34326
34327
34328
34329
34330
34334
34335
34336
34337
34340
34341
34342
34359
34361
34367
34368
34369
34370
34372
34374
34375
34376
34377
34379
34380
34381
34384
34387
34388
34389
34390
35001
35002
35003
35004
35005
35006
35007
35008
35009
35010
35011
35016
35017
35018
35019
35020
35021
35022
35023
35024
35025
35026
35027
35028
35030
35031
35032
35033
35035
35038
35040
35043
35044
35046
35047
35048
35049
35050
35051
35052
35053
35054
35055
3505

40767
40768
40769
40770
40771
40772
40773
40774
40775
40776
40777
40778
40779
40781
40782
40783
40784
40785
40786
40787
40788
40789
40790
40791
40792
40793
40794
40796
40797
40798
40799
40800
41401
41402
41404
41405
41407
41409
41411
41413
41415
41416
41417
41419
41420
41421
41422
41423
41424
41425
41426
41427
41428
41429
41430
41431
41432
41434
41436
41438
41439
41440
41441
41443
41444
41445
41446
41447
41449
41452
41453
41454
41455
41456
41457
41458
41459
41460
41461
41462
41463
41464
41465
41466
41467
41468
41469
41470
41471
41472
41473
41474
41475
41477
41478
41479
41480
41481
41482
41483
41484
41485
41486
41487
41488
41490
41491
41492
41493
41495
41496
41497
41498
41499
41500
41503
41504
41505
41507
41508
41511
41513
41515
41516
41517
41518
41520
41521
41523
41524
41526
41527
41528
41529
41530
41531
41532
41533
41534
41535
41536
41537
41538
41539
41540
41541
41542
41543
41544
41545
41546
41547
41548
41549
41550
41551
41552
41553
41554
41555
41556
41558
41559
41560
41561
41562
4156

In [160]:
tmdb_df.shape

(1200, 25)

In [153]:
((467223/40)*10) / 3600

32.446041666666666

In [154]:
3600 /3 


1200.0

In [161]:
# 1200 -> ~1MB
467223/1200
# -> 400MB

389.3525